# Prediction of Wine Quality - My Method

### Parameters present in wine-quality csv files:
#### 1. Fixed Acidity <br> 2. Volatile Acidity <br> 3. Citric Acid <br> 4. Residual Sugar <br> 5. Chlorides <br> 6. Free Sulfur Dioxide <br> 7. Total Sulfur Dioxide <br> 8. Density <br> 9. pH <br> 10. Sulphates <br> 11. Alcohol Percentage (%) <br> 12. Quality

## Preliminary Data Exploration

In [2]:
import pandas as pd
import numpy as np

In [3]:
# We start with White Wine
df = pd.read_csv("winequality-white.csv")

In [4]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


### Summary:<br>We have 12 columns including the quality. From the looks of it, all the parameters are continuous. Linear Regression could be used as a good preliminary test for setting a baseline. 

## Linear Regression - Prediction of Quality using the Parameters

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
df_X = df.copy()
df_X = df_X.drop(["quality"],axis=1)
df_y = df['quality']

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=1)

linear_regression1 = LinearRegression()
linear_regression1.fit(X_train, y_train)

y_pred = linear_regression1.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error of:",mse,"\nR2 Square:",r2)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')
print(f'Accuracy based on MAPE: {100-mape:.2f}%')

Mean Squared Error of: 0.542548726356103 
R2 Square: 0.2901221946674388
Mean Absolute Percentage Error (MAPE): 10.20%
Accuracy based on MAPE: 89.80%


### Summary:<br> Based on LinearRegression we have received a prediction accuracy of 89.8%.

## Linear Regression - Feature Importance

In [7]:
linear_regression2 = LinearRegression()
linear_regression2.fit(df_X, df_y)
feature_importance = pd.DataFrame({'Feature': df_X.columns, 'Coefficient': linear_regression2.coef_})
feature_importance = feature_importance.sort_values(by='Coefficient', ascending=False)
print(feature_importance)

                 Feature  Coefficient
8                     pH     0.686344
9              sulphates     0.631476
10               alcohol     0.193476
3         residual sugar     0.081483
0          fixed acidity     0.065520
2            citric acid     0.022090
5    free sulfur dioxide     0.003733
6   total sulfur dioxide    -0.000286
4              chlorides    -0.247277
1       volatile acidity    -1.863177
7                density  -150.284181


### Summary:<br>pH and amount of Sulphates in the alcohol seems to have a very high correlation with the quality of alcohol. Density on the other hand is very negatively correlated to the quality of the alcohol.

## Feature Engineering based on Binning the Data into:<br> 1. Bad (0-5 Inc.) <br> 2. Good (6-10 Inc.)

In [8]:
df['Quality Type'] = ['Bad' if x<6 else 'Good' for x in df['quality']]

### Using Logistic Regression for Quality Prediction

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr = LogisticRegression()

df_X = df.copy()
df_X = df_X.drop(["Quality Type"], axis=1)
df_X = df_X.drop(["quality"], axis=1)
df_y = df['Quality Type']

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=1)

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 74.39%


/Users/geolangsatnarzary/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## We extend the same analysis for Red-Wine Quality Prediction - Prof's Method

In [10]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [12]:
df['good_bad'] = pd.cut(x=df.quality, bins=[0,6,10], labels=[0,1])
df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,good_bad
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,0
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,0
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,1
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,1
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,0


### Feature preparation

In [ ]:
X = df.iloc[:,:-2]
y = df.iloc[:, -1]
y.head(10)

### Data Normalizing

In [17]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
x_transformed = ss.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(x_transformed, y, test_size=0.3, random_state=0)

### Classification

In [18]:
from sklearn.svm import SVC

clf = SVC(kernel="linear", random_state=0)
clf.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

In [19]:
y_pred = clf.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[410  20]
 [ 35  15]]


In [25]:
print("Accuracy Score:\n", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy Score:
 0.8854166666666666
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.95      0.94       430
           1       0.43      0.30      0.35        50

    accuracy                           0.89       480
   macro avg       0.67      0.63      0.65       480
weighted avg       0.87      0.89      0.88       480
